# Récupération du nombre d'Occurrence 

## instal et ouvrir graphDB

In [1]:
from rdflib import ConjunctiveGraph
from SPARQLWrapper import SPARQLWrapper, JSON
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import numpy as np
from IPython.display import display, HTML



import pandas as pd

def remoteQuery(query, endpoint):
    endpoint.setQuery(query)
    try:
        result = endpoint.queryAndConvert()
        pd.set_option("display.max_rows",None,"display.max_colwidth",6000,"display.width",6000,)
        df = pd.DataFrame(result['results']['bindings'])
        df = df.applymap(lambda x: x['value'])
        return df
        #return (result['results']['bindings'])
    except Exception as e:
        print(e)

In [2]:
ep_biotools = SPARQLWrapper("http://localhost:7200/repositories/Project25")
ep_biotools.setReturnFormat(JSON)

# FORMAT

## liste format input

In [212]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format (COUNT(?format) as ?count) ?label
   
WHERE
{
    ?x bsc:input [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .   # Neeeded because the old data with URIs as literals for Data and Formats
    
}
GROUP BY ?format ?label
ORDER BY ASC(?format)
#LIMIT 8
"""

In [213]:
print(f"Number of EDAM formats use as input in bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

Number of EDAM formats use as input in bio.tools: 272


,format,count,label
0,http://edamontology.org/format_1196,80,SMILES
1,http://edamontology.org/format_1197,5,InChI
2,http://edamontology.org/format_1199,5,InChIKey
3,http://edamontology.org/format_1207,25,nucleotide
4,http://edamontology.org/format_1208,63,protein
5,http://edamontology.org/format_1209,10,consensus
6,http://edamontology.org/format_1210,20,pure nucleotide
7,http://edamontology.org/format_1211,15,unambiguous pure nucleotide
8,http://edamontology.org/format_1212,20,dna
9,http://edamontology.org/format_1213,23,rna


## liste format output

In [5]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format (COUNT(?format) as ?count) ?label
   
WHERE
{
    ?tool bsc:output [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .   # Neeeded because the old data with URIs as literals for Data and Formats
}
GROUP BY ?format ?label
ORDER BY ASC(?format)
#LIMIT 8
"""

In [6]:
print(f"Number of EDAM formats use as output in bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

Number of EDAM formats use as output in bio.tools: 232


,format,count,label
0,http://edamontology.org/format_1196,2,SMILES
1,http://edamontology.org/format_1208,1,protein
2,http://edamontology.org/format_1212,1,dna
3,http://edamontology.org/format_1332,2,FASTA search results format
4,http://edamontology.org/format_1333,9,BLAST results
5,http://edamontology.org/format_1360,15,meme-motif
6,http://edamontology.org/format_1367,1,JASPAR format
7,http://edamontology.org/format_1369,1,MEME background Markov model
8,http://edamontology.org/format_1423,2,Phylip distance matrix
9,http://edamontology.org/format_1445,1,Phylip tree distance format


## liste format

In [7]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format (COUNT(?format) as ?count) ?label
   
WHERE{
{
    ?tool bsc:output [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .   # Neeeded because the old data with URIs as literals for Data and Formats
}UNION {

?tool bsc:input [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .   # Neeeded because the old data with URIs as literals for Data and Formats
}
}
GROUP BY ?format ?label
ORDER BY ASC(?format)
#LIMIT 8
"""

In [8]:
print(f"Number of Bio.tools entries annotated with EDAM formats (as output): {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

Number of Bio.tools entries annotated with EDAM formats (as output): 320


,format,count,label
0,http://edamontology.org/format_1196,18,SMILES
1,http://edamontology.org/format_1197,1,InChI
2,http://edamontology.org/format_1199,1,InChIKey
3,http://edamontology.org/format_1207,5,nucleotide
4,http://edamontology.org/format_1208,14,protein
5,http://edamontology.org/format_1209,2,consensus
6,http://edamontology.org/format_1210,4,pure nucleotide
7,http://edamontology.org/format_1211,3,unambiguous pure nucleotide
8,http://edamontology.org/format_1212,6,dna
9,http://edamontology.org/format_1213,5,rna


## How many OBSOLETE EDAM formats are used to annotate bio.tools?

In [8]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?format) as ?count)
    ?label
WHERE
{
    ?param
        rdf:type bsct:FormalParameter ;
        sc:encodingFormat ?format .
    ?format rdfs:label ?label .
    ?format owl:deprecated true .
}
GROUP BY ?format ?label
ORDER BY DESC(?count)
#ORDER BY ASC(?format)
"""

In [9]:
print(f"Number of OBSOLETE EDAM formats used in Bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

Number of OBSOLETE EDAM formats used in Bio.tools: 0


""


# DATA

## Which EDAM data are used for bio.tools annotation?

In [9]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>

SELECT DISTINCT ?data (COUNT(?data) as ?count) ?label
WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data]. 
   ?data rdfs:label ?label .
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data]. 
   ?data rdfs:label ?label .
    }
}
GROUP BY ?data ?label
ORDER BY ASC(?data)
"""


In [10]:
print(f"EDAM data used to annotate bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)

display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

EDAM data used to annotate bio.tools: 570


,data,count,label
0,http://edamontology.org/data_0006,1080,Data
1,http://edamontology.org/data_0581,1,Database
2,http://edamontology.org/data_0582,24,Ontology
3,http://edamontology.org/data_0842,106,Identifier
4,http://edamontology.org/data_0844,10,Molecular mass
5,http://edamontology.org/data_0845,1,Molecular charge
6,http://edamontology.org/data_0846,9,Chemical formula
7,http://edamontology.org/data_0848,12,Raw sequence
8,http://edamontology.org/data_0849,193,Sequence record
9,http://edamontology.org/data_0850,68,Sequence set


## Which deprecated EDAM data are used for bio.tools annotation?

In [5]:
q= """
PREFIX edam:<http://edamontology.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>

PREFIX sc: <http://schema.org/>
SELECT DISTINCT ?data (COUNT(?data) as ?count) ?label WHERE {
    {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:input [sc:additionalType ?data].
   ?data rdfs:subClassOf <http://www.w3.org/2002/07/owl#DeprecatedClass> . 
   ?data rdfs:label ?label .
    } UNION {
?biotools_id rdf:type <http://schema.org/SoftwareApplication> ;
   bsc:output [sc:additionalType ?data].        
   ?data rdfs:subClassOf <http://www.w3.org/2002/07/owl#DeprecatedClass> . 
   ?data rdfs:label ?label .
    }
}
GROUP BY ?data ?label
ORDER BY ASC(?data)
"""
print(f"EDAM data used to annotate bio.tools:{len(remoteQuery(query=q, endpoint=ep_biotools))}")
remoteQuery(query=q, endpoint=ep_biotools)

EDAM data used to annotate bio.tools:19


,data,count,label
0,http://edamontology.org/data_0581,1,Database
1,http://edamontology.org/data_0848,12,Raw sequence
2,http://edamontology.org/data_0868,2,Profile-profile alignment
3,http://edamontology.org/data_0869,6,Sequence-profile alignment
4,http://edamontology.org/data_0953,2,Version information
5,http://edamontology.org/data_1057,1,Sequence database name
6,http://edamontology.org/data_1674,1,Sequence database cross-references
7,http://edamontology.org/data_1864,2,Map set data
8,http://edamontology.org/data_2086,1,Nucleic acid structure data
9,http://edamontology.org/data_2751,1,GenomeReviews ID


# TOPIC

## How many EDAM topics are used to annotate bio.tools?

In [3]:
q= """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT  DISTINCT?topic ?label (COUNT(?topic) as ?count) WHERE {
?x rdf:type <http://schema.org/SoftwareApplication> ;
   <http://schema.org/name> ?name ; 
   <http://schema.org/applicationSubCategory> ?topic .
   ?topic rdfs:label ?label .
} GROUP BY ?topic ?label
ORDER BY DESC(?count)
"""

In [4]:
print(f"nb of EDAM topics used in bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

nb of EDAM topics used in bio.tools: 263


,topic,label,count
0,http://edamontology.org/topic_3474,Machine learning,3050
1,http://edamontology.org/topic_0602,"Molecular interactions, pathways and networks",2717
2,http://edamontology.org/topic_0203,Gene expression,2665
3,http://edamontology.org/topic_0154,Small molecules,2427
4,http://edamontology.org/topic_3170,RNA-Seq,2341
5,http://edamontology.org/topic_0080,Sequence analysis,2220
6,http://edamontology.org/topic_3168,Sequencing,2165
7,http://edamontology.org/topic_0625,Genotype and phenotype,1678
8,http://edamontology.org/topic_0121,Proteomics,1672
9,http://edamontology.org/topic_3512,Gene transcripts,1609


## topic obsolète

In [6]:
q= """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT  DISTINCT?topic ?label WHERE {
?tool rdf:type <http://schema.org/SoftwareApplication> ; 
   <http://schema.org/applicationSubCategory> ?topic.
   ?topic rdfs:label ?label.
?topic rdfs:subClassOf <http://www.w3.org/2002/07/owl#DeprecatedClass>.

} GROUP BY ?topic?label
"""

In [7]:
print(f"nb of EDAM topics used in bio.tools: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

nb of EDAM topics used in bio.tools: 6


,topic,label
0,http://edamontology.org/topic_3041,Sequence databases
1,http://edamontology.org/topic_3521,2D PAGE experiment
2,http://edamontology.org/topic_0137,Protein hydropathy
3,http://edamontology.org/topic_3043,Protein sequences
4,http://edamontology.org/topic_0083,Alignment
5,http://edamontology.org/topic_0644,Proteome


# OPERATION

## How many EDAM operations are used to annotate bio.tools?

In [5]:
q= """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sc: <http://schema.org/>

SELECT DISTINCT ?operation ?label (COUNT(?operation) as ?count) WHERE {

?x rdf:type sc:SoftwareApplication ;
   sc:featureList ?operation .
   ?operation rdfs:label ?label . 
} 
GROUP BY ?operation ?label
"""


In [6]:
#
results1=remoteQuery(query=q, endpoint=ep_biotools)
print(f"nb of edam operation used in bio.tools: {len(results1)}")
#print(results1)

display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results1.to_html() +
             "</div>"))


nb of edam operation used in bio.tools: 546


,operation,label,count
0,http://edamontology.org/operation_0306,Text mining,239
1,http://edamontology.org/operation_3431,Deposition,821
2,http://edamontology.org/operation_3196,Genotyping,1408
3,http://edamontology.org/operation_3202,Polymorphism detection,91
4,http://edamontology.org/operation_2944,Physical mapping,27
5,http://edamontology.org/operation_2121,Sequence file editing,13
6,http://edamontology.org/operation_3282,ID mapping,63
7,http://edamontology.org/operation_3504,Variant pattern analysis,21
8,http://edamontology.org/operation_0335,Formatting,493
9,http://edamontology.org/operation_2479,Protein sequence analysis,493


## How many bio.tools entries are annotated with deprecated operations?

In [3]:
q3="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sc: <http://schema.org/>

SELECT DISTINCT ?operation ?label WHERE {
?x rdf:type sc:SoftwareApplication ;
   sc:featureList ?operation .
   ?operation rdfs:label ?label .
?operation rdfs:subClassOf <http://www.w3.org/2002/07/owl#DeprecatedClass> .   

} 
GROUP BY ?operation ?label
"""

In [4]:
results3=remoteQuery(query=q3, endpoint=ep_biotools)
print(f"number of bio.tools entries annotated with deprecated operation: {len(results3)}")
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results3.to_html() +
             "</div>"))

number of bio.tools entries annotated with deprecated operation: 20


,operation,label
0,http://edamontology.org/operation_3202,Polymorphism detection
1,http://edamontology.org/operation_2414,Protein function analysis
2,http://edamontology.org/operation_3083,Pathway or network visualisation
3,http://edamontology.org/operation_3439,Pathway or network prediction
4,http://edamontology.org/operation_3562,Network simulation
5,http://edamontology.org/operation_0277,Pathway or network comparison
6,http://edamontology.org/operation_2497,Pathway or network analysis
7,http://edamontology.org/operation_0271,Structure prediction
8,http://edamontology.org/operation_2513,Sequence generation (nucleic acid)
9,http://edamontology.org/operation_3441,Plotting


In [ ]:
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

# PROFONDEUR

In [ ]:
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

## FORMAT

### racine format, test

In [127]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?tool
    (COUNT(?format) as ?count)
    
WHERE
{
    
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1915] .
    }
    FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_1360] .
    }
    FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_1476] .
    }
    FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_1929] .
    }
     FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_2200] .
    }
    FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_2330] .
    }
     FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_2333] .
    }
    FILTER NOT EXISTS
    {
       ?tool ?param [sc:encodingFormat edam:format_3475] .
    }
}
GROUP BY ?format
ORDER BY ASC(?format)
#LIMIT 8
"""

In [128]:
print(f"Number of Bio.tools entries annotated only with the root 'Format' concept (input or output): {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

Number of Bio.tools entries annotated with the root 'Format' concept (input or output): 1


,format,count
0,http://edamontology.org/format_1915,6


## les 10 plus important

### TEXTUAL FORMAT

In [203]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
   ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [204]:
print(f"'textual Format': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'textual Format': 136


,format,count
0,http://edamontology.org/format_2330,1497
1,http://edamontology.org/format_3547,199
2,http://edamontology.org/format_1929,147
3,http://edamontology.org/format_3475,129
4,http://edamontology.org/format_3003,104
5,http://edamontology.org/format_2331,100
6,http://edamontology.org/format_3016,96
7,http://edamontology.org/format_2332,74
8,http://edamontology.org/format_1975,53
9,http://edamontology.org/format_2305,50


In [205]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
   ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [206]:
print(f"'textual Format' sans rang1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'textual Format' sans rang1: 84


,format,count
0,http://edamontology.org/format_2330,1121
1,http://edamontology.org/format_3547,188
2,http://edamontology.org/format_2331,64
3,http://edamontology.org/format_2333,35
4,http://edamontology.org/format_3508,35
5,http://edamontology.org/format_3033,21
6,http://edamontology.org/format_3003,18
7,http://edamontology.org/format_1476,17
8,http://edamontology.org/format_3603,16
9,http://edamontology.org/format_3016,16


In [207]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
   ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    
     FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
  FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3033] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3621] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3244] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3706] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2058] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3326] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2195] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3814] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2033] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1920] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3650] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2013] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3617] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2549] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1921] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2196] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2548] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2306] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3284] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2350] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3008] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1963] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1984] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1919] .
    }.FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1333] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3578] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2921] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2035] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3549] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3681] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1196] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3467] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2068] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2554] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [208]:
print(f"'textual Format' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'textual Format' seul: 1


,format,count
0,http://edamontology.org/format_2330,680


### FASTA

In [191]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [192]:
print(f"'FASTA': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'FASTA': 144


,format,count
0,http://edamontology.org/format_1929,990
1,http://edamontology.org/format_2330,207
2,http://edamontology.org/format_1930,112
3,http://edamontology.org/format_3016,97
4,http://edamontology.org/format_3475,94
5,http://edamontology.org/format_2572,87
6,http://edamontology.org/format_3003,66
7,http://edamontology.org/format_2331,44
8,http://edamontology.org/format_2305,32
9,http://edamontology.org/format_1476,31


In [195]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    
   
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [196]:
print(f"'FASTA' sans rang 1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'FASTA' sans rang 1: 96


,format,count
0,http://edamontology.org/format_1929,720
1,http://edamontology.org/format_1930,78
2,http://edamontology.org/format_1476,26
3,http://edamontology.org/format_3651,23
4,http://edamontology.org/format_1936,20
5,http://edamontology.org/format_2331,16
6,http://edamontology.org/format_3654,15
7,http://edamontology.org/format_3244,15
8,http://edamontology.org/format_2305,13
9,http://edamontology.org/format_3652,13


In [126]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1936] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2305] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3654] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3244] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1912] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3160] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1982] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2182] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2546] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1915] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1957] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3466] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2306] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3655] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3976] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3621] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1939] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1961] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3585] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2078] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2031] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1457] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3652] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2068] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2205] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2064] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3617] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [128]:
print(f"'FASTA' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

Number of Bio.tools entries annotated only with 'FASTA' concept (input or output): 1


,format,count
0,http://edamontology.org/format_1929,470


### HTLM

In [185]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    
    
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [186]:
print(f"'HTML': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'HTML': 93


,format,count
0,http://edamontology.org/format_2331,635
1,http://edamontology.org/format_2330,86
2,http://edamontology.org/format_2332,64
3,http://edamontology.org/format_3464,58
4,http://edamontology.org/format_3508,37
5,http://edamontology.org/format_3475,36
6,http://edamontology.org/format_1929,36
7,http://edamontology.org/format_3752,22
8,http://edamontology.org/format_3603,19
9,http://edamontology.org/format_3547,19


In [187]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [188]:
print(f"'HTML'sans rang1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'HTML'sans rang1: 34


,format,count
0,http://edamontology.org/format_2331,413
1,http://edamontology.org/format_3464,16
2,http://edamontology.org/format_3508,12
3,http://edamontology.org/format_2532,4
4,http://edamontology.org/format_3547,4
5,http://edamontology.org/format_1476,4
6,http://edamontology.org/format_3603,3
7,http://edamontology.org/format_3617,3
8,http://edamontology.org/format_3619,3
9,http://edamontology.org/format_3655,3


In [189]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
    
    
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1936] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2305] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3654] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3244] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1912] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3160] .
    }.
    
    
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1982] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2182] .
    }.
   
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2546] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1915] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1957] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3466] .
    }.
  
  
      FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2306] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3655] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    
    
    
    
   FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3617] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2350] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2532] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
    
    
      FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1984] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2310] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2058] .
    }.
    
     FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1998] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1973] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3814] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1207] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1333] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3747] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1978] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3166] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3477] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [190]:
print(f"'HTML' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'HTML' seul: 1


,format,count
0,http://edamontology.org/format_2331,354


### TSV

In [177]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    
   
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [178]:
print(f"'TSV': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'TSV': 98


,format,count
0,http://edamontology.org/format_3475,633
1,http://edamontology.org/format_2330,125
2,http://edamontology.org/format_3016,83
3,http://edamontology.org/format_1929,67
4,http://edamontology.org/format_3003,66
5,http://edamontology.org/format_2572,36
6,http://edamontology.org/format_3752,32
7,http://edamontology.org/format_2331,31
8,http://edamontology.org/format_3547,31
9,http://edamontology.org/format_3464,28


In [181]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
   
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [182]:
print(f"'TSV' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'TSV' seul: 63


,format,count
0,http://edamontology.org/format_3475,418
1,http://edamontology.org/format_3547,25
2,http://edamontology.org/format_2331,19
3,http://edamontology.org/format_3464,13
4,http://edamontology.org/format_3003,13
5,http://edamontology.org/format_3508,12
6,http://edamontology.org/format_3603,11
7,http://edamontology.org/format_3033,10
8,http://edamontology.org/format_3016,7
9,http://edamontology.org/format_3247,6


In [175]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3033] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3247] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1936] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2305] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3654] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1936] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2305] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3654] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2013] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3020] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3619] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3751] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2058] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3621] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2197] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3604] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3005] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3591] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1419] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3713] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3652] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3466] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1637] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1637] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1638] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1639] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3015] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3702] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3758] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3788] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3579] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3244] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3327] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3615] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3916] .
    }.
}
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [176]:
print(f"'TSV' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'TSV' seul: 1


,format,count
0,http://edamontology.org/format_3475,278


###  IMAGE FORMAT 

In [169]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
    
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [170]:
print(f"'IMAGE FORMAT ' sans rang1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'IMAGE FORMAT ' sans rang1: 17


,format,count
0,http://edamontology.org/format_3547,197
1,http://edamontology.org/format_3033,39
2,http://edamontology.org/format_1975,8
3,http://edamontology.org/format_2013,5
4,http://edamontology.org/format_2331,4
5,http://edamontology.org/format_3468,3
6,http://edamontology.org/format_2006,3
7,http://edamontology.org/format_2333,2
8,http://edamontology.org/format_2078,2
9,http://edamontology.org/format_3620,2


In [171]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
    
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [172]:
print(f"'IMAGE FORMAT ': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'IMAGE FORMAT ': 37


,format,count
0,http://edamontology.org/format_3547,430
1,http://edamontology.org/format_2330,194
2,http://edamontology.org/format_3033,48
3,http://edamontology.org/format_3475,35
4,http://edamontology.org/format_2331,20
5,http://edamontology.org/format_3508,16
6,http://edamontology.org/format_1975,9
7,http://edamontology.org/format_3579,9
8,http://edamontology.org/format_3466,7
9,http://edamontology.org/format_1929,6


In [173]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3033] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2006] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2078] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3579] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3591] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2058] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2013] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3166] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3617] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [174]:
print(f"'IMAGE FORMAT ' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'IMAGE FORMAT ' seul: 1


,format,count
0,http://edamontology.org/format_3547,129


### BAM

In [161]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
   
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [162]:
print(f"'BAM ': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'BAM ': 46


,format,count
0,http://edamontology.org/format_2572,282
1,http://edamontology.org/format_3016,95
2,http://edamontology.org/format_1929,66
3,http://edamontology.org/format_3003,65
4,http://edamontology.org/format_2573,63
5,http://edamontology.org/format_2330,61
6,http://edamontology.org/format_3475,48
7,http://edamontology.org/format_1930,27
8,http://edamontology.org/format_3327,13
9,http://edamontology.org/format_2305,12


In [167]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [168]:
print(f"'BAM 'sans rang1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'BAM 'sans rang1: 21


,format,count
0,http://edamontology.org/format_2572,131
1,http://edamontology.org/format_3016,30
2,http://edamontology.org/format_3003,21
3,http://edamontology.org/format_3006,8
4,http://edamontology.org/format_1930,7
5,http://edamontology.org/format_3327,6
6,http://edamontology.org/format_2305,5
7,http://edamontology.org/format_3462,3
8,http://edamontology.org/format_2306,2
9,http://edamontology.org/format_2331,2


In [165]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3327] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3006] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2546] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2306] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3585] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2545] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3751] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3009] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3462] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [166]:
print(f"'BAM 'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'BAM 'seul: 1


,format,count
0,http://edamontology.org/format_2572,63


### VCF

In [154]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [155]:
print(f"'VCF ': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'VCF ': 46


,format,count
0,http://edamontology.org/format_3016,251
1,http://edamontology.org/format_2330,86
2,http://edamontology.org/format_2572,78
3,http://edamontology.org/format_3475,56
4,http://edamontology.org/format_3003,53
5,http://edamontology.org/format_1929,41
6,http://edamontology.org/format_3752,9
7,http://edamontology.org/format_3020,7
8,http://edamontology.org/format_2331,6
9,http://edamontology.org/format_2573,6


In [156]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
    
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [157]:
print(f"'VCF 'sans rang 1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'VCF 'sans rang 1: 14


,format,count
0,http://edamontology.org/format_3016,90
1,http://edamontology.org/format_3003,5
2,http://edamontology.org/format_3020,4
3,http://edamontology.org/format_1975,2
4,http://edamontology.org/format_2331,2
5,http://edamontology.org/format_3033,2
6,http://edamontology.org/format_3464,1
7,http://edamontology.org/format_1964,1
8,http://edamontology.org/format_3286,1
9,http://edamontology.org/format_2306,1


In [158]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3020] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3033] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3286] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3499] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3019] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [160]:
print(f"'VCF 'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'VCF 'seul: 1


,format,count
0,http://edamontology.org/format_3016,70


### CSV

In [148]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.

  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [149]:
print(f"'CSV ': {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'CSV ': 89


,format,count
0,http://edamontology.org/format_3752,236
1,http://edamontology.org/format_1929,41
2,http://edamontology.org/format_3475,31
3,http://edamontology.org/format_2331,29
4,http://edamontology.org/format_2330,26
5,http://edamontology.org/format_3603,25
6,http://edamontology.org/format_2572,23
7,http://edamontology.org/format_3508,21
8,http://edamontology.org/format_3620,21
9,http://edamontology.org/format_3604,21


In [152]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
         FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.

  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [153]:
print(f"'CSV 'sans rang 1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'CSV 'sans rang 1: 45


,format,count
0,http://edamontology.org/format_3752,162
1,http://edamontology.org/format_2331,25
2,http://edamontology.org/format_3603,18
3,http://edamontology.org/format_3604,14
4,http://edamontology.org/format_3508,13
5,http://edamontology.org/format_3620,10
6,http://edamontology.org/format_3579,8
7,http://edamontology.org/format_3464,7
8,http://edamontology.org/format_3654,6
9,http://edamontology.org/format_3244,6


In [146]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3654] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3604] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3712] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3591] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1637] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3751] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3746] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3650] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2554] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3714] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3652] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [147]:
print(f"'CSV 'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'CSV 'seul: 1


,format,count
0,http://edamontology.org/format_3752,74


### PDB

In [140]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [141]:
print(f"'PDB ' sans rang 1 : {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'PDB ' sans rang 1 : 29


,format,count
0,http://edamontology.org/format_1476,178
1,http://edamontology.org/format_1477,18
2,http://edamontology.org/format_3816,10
3,http://edamontology.org/format_3814,6
4,http://edamontology.org/format_1637,5
5,http://edamontology.org/format_3464,5
6,http://edamontology.org/format_2331,5
7,http://edamontology.org/format_3650,4
8,http://edamontology.org/format_3875,4
9,http://edamontology.org/format_3750,4


In [142]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [143]:
print(f"liste format associé à 'PDB ' concept (input or output): {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

liste format associé à 'PDB ' concept (input or output): 59


,format,count
0,http://edamontology.org/format_1476,236
1,http://edamontology.org/format_1929,28
2,http://edamontology.org/format_1477,22
3,http://edamontology.org/format_2330,20
4,http://edamontology.org/format_3464,11
5,http://edamontology.org/format_3816,10
6,http://edamontology.org/format_2331,7
7,http://edamontology.org/format_3650,6
8,http://edamontology.org/format_3875,6
9,http://edamontology.org/format_3814,6


In [144]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1477] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3816] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3650] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3875] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3910] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1952] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3866] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2554] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_4002] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1957] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2033] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3814] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2187] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2546] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1637] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3877] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [145]:
print(f"'PDB ' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'PDB ' seul: 1


,format,count
0,http://edamontology.org/format_1476,121


### FASTAQ

In [138]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
     
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [139]:
print(f"liste des format associé à 'FASTAQ' concept (input or output): {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

liste des format associé à 'FASTAQ' concept (input or output): 53


,format,count
0,http://edamontology.org/format_1930,229
1,http://edamontology.org/format_1929,114
2,http://edamontology.org/format_2572,37
3,http://edamontology.org/format_2573,17
4,http://edamontology.org/format_2330,17
5,http://edamontology.org/format_3475,11
6,http://edamontology.org/format_1931,11
7,http://edamontology.org/format_3016,10
8,http://edamontology.org/format_1932,9
9,http://edamontology.org/format_1933,8


In [136]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3590] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3245] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
     FILTER NOT EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [137]:
print(f"'FASTAQ'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'FASTAQ'seul: 1


,format,count
0,http://edamontology.org/format_1930,72


In [84]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
     FILTER not EXISTs
    {
        {
        ?tool ?param [sc:encodingFormat edam:format_2333] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_2350] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_2331] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_3464] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_2376] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_2330] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_2332] .
        }
        UNION{
        ?tool ?param [sc:encodingFormat edam:format_3750] .
        }
    }.
   
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [85]:
print(f"'FASTAQ' sans rang1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'FASTAQ' sans rang1: 31


,format,count
0,http://edamontology.org/format_1930,202
1,http://edamontology.org/format_1929,97
2,http://edamontology.org/format_2572,34
3,http://edamontology.org/format_2573,14
4,http://edamontology.org/format_3016,10
5,http://edamontology.org/format_1931,9
6,http://edamontology.org/format_3475,6
7,http://edamontology.org/format_1932,6
8,http://edamontology.org/format_1933,6
9,http://edamontology.org/format_2305,4


In [82]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
     FILTER not EXISTs
    {
        { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
        }.
         FILTER not EXISTs
        {
            {
            ?tool ?param [sc:encodingFormat edam:format_2333] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_2350] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_2331] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_3464] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_2376] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_2330] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_2332] .
            }
            UNION{
            ?tool ?param [sc:encodingFormat edam:format_3750] .
            }
        }.
    }.
   
  }
GROUP BY ?format
ORDER BY DESC(?count)
#LIMIT 8
"""

In [83]:
print(f"'FASTAQ' avec rang1: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'FASTAQ' sans rang1: 34


,format,count
0,http://edamontology.org/format_1930,27
1,http://edamontology.org/format_2330,17
2,http://edamontology.org/format_1929,17
3,http://edamontology.org/format_2332,6
4,http://edamontology.org/format_1925,6
5,http://edamontology.org/format_1942,6
6,http://edamontology.org/format_1927,5
7,http://edamontology.org/format_3475,5
8,http://edamontology.org/format_1210,4
9,http://edamontology.org/format_2573,3


In [ ]:
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

## RANG1


In [ ]:
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

### textual Format
dejà fait

In [164]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
   ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    
     FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER NOT EXISTS
    { 
       ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
    FILTER NOT EXISTS
    { 
       ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3016] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3033] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3621] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3244] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1930] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3706] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2058] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3326] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2195] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3814] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2033] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1920] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2573] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3650] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2013] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3617] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2549] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1921] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2196] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2548] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2306] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3284] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2350] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3008] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1963] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1984] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1919] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1333] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2921] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2035] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3549] .
    }.
  
      FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3681] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1196] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    
   FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3467] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2068] .
    }.
     FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3578] .
    }.
    
    
    
     
     
  
    
   
}
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [165]:
print(f"'textual Format' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'textual Format' seul: 2


,format,count,label
0,http://edamontology.org/format_2330,684,Textual format
1,http://edamontology.org/format_1930,6,FASTQ


### HTLM
dejà fait

In [179]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
     ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.

    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2572] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER NOT EXISTS
    { 
       ?tool ?param [sc:encodingFormat edam:format_1476] .
    }.
    
    
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1936] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2305] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3654] .
    }.
 
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3244] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1912] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2200] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3160] .
    }.
    
    
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1931] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1982] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1932] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3003] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1933] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2182] .
    }.
   
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2546] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1915] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1957] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3466] .
    }.
  
  
      FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2306] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3655] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    
    
    
    
   FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3617] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2350] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2532] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3547] .
    }.
    
    
      FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1984] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2310] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_2058] .
    }.
    
     FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1998] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1973] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3814] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1207] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1333] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3747] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_1978] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3166] .
    }.
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3477] .
    }.
    
    FILTER NOT EXISTS
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.
}
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [180]:
print(f"'HTML' seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'HTML' seul: 1


,format,count,label
0,http://edamontology.org/format_2331,354,HTML


### XML

In [187]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1360] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1915] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3604] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1925] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3713] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3255] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1920] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3156] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3681] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3710] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3247] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3711] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3162] .
    }.
 
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3750] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1912] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3159] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3160] .
    }.
    FILTER not EXISTs
    
    
    
    { 
        ?tool ?param [sc:encodingFormat edam:format_2350] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1975] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1963] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3468] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3651] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3652] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3834] .
    }.
  }
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [188]:
print(f"'XML'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'XML'seul: 1


,format,count,label
0,http://edamontology.org/format_2332,26,XML


### JSON

In [199]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
    
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    
    
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
   
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1360] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3752] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3475] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1915] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3604] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3603] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1925] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3713] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3255] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1920] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3156] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3681] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3710] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3247] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3711] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3162] .
    }.
   
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2000] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3750] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1910] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1912] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3159] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3160] .
    }.
    
    
    
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1957] .
    }.
   
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2305] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3158] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1921] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1964] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3242] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2062] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2548] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_4002] .
    }.
     FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3508] .
    }.
     
  }
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [200]:
print(f"'JSON'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'JSON'seul: 1


,format,count,label
0,http://edamontology.org/format_3464,22,JSON


### Binary format

In [87]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
 FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    # FILTER EXISTs
    #{ 
    #    ?tool ?param [sc:encodingFormat edam:format_3475] .
    #}.
     FILTER EXISTs
    { 
       ?tool ?param [sc:encodingFormat edam:format_1929] .
    }.
     FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    
  }
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [88]:
print(f"'Binary format'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'Binary format'seul: 0


""


### RDF Format

In [79]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2376] .
    }.
    #FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
     FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
     FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
     FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3751] .
    }.
   
   
   FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2545] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3773] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3750] .
    }.
  }
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [80]:
print(f"'RDF format'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'RDF format'seul: 3


,format,count,label
0,http://edamontology.org/format_2331,4,HTML
1,http://edamontology.org/format_2376,3,RDF format
2,http://edamontology.org/format_3752,1,CSV


###	Format (by type of data)

In [83]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2350] .
    }.
   FILTER not EXISTs
    { 
       ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
   FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2331] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2333] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3000] .
    }.
    
  }
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [84]:
print(f"'Format (by type of data)'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'Format (by type of data)'seul: 1


,format,count,label
0,http://edamontology.org/format_2350,4,Format (by type of data)


### YAML

In [25]:
q= """
PREFIX biotools: <https://bio.tools/ontology/>
PREFIX bsc: <http://bioschemas.org/>
PREFIX bsct: <http://bioschemas.org/types/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX edam: <http://edamontology.org/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX sc: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?format
    (COUNT(?tool) as ?count)
    ?label
WHERE
{
    ?tool ?param [sc:encodingFormat ?format] .
    ?format rdfs:label ?label .
   FILTER EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3750] .
    }.
    
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3464] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2585] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3620] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_1957] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_2332] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3033] .
    }.
    FILTER not EXISTs
    { 
        ?tool ?param [sc:encodingFormat edam:format_3773] .
    }.
   FILTER not EXISTs
   { 
        ?tool ?param [sc:encodingFormat edam:format_2330] .
    }.
  }
GROUP BY ?format ?label
ORDER BY DESC(?count)
#LIMIT 8
"""

In [26]:
print(f"'YAML'seul: {len(remoteQuery(query=q, endpoint=ep_biotools))}")
results=remoteQuery(query=q, endpoint=ep_biotools)
display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
             results.to_html() +
             "</div>"))

'YAML'seul: 1


,format,count,label
0,http://edamontology.org/format_3750,1,YAML
